# Experimentos

In [1]:
# imports
import numpy as np
import pandas as pd

# time
import time
from datetime import timedelta

# Metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

# ETL
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Oversampling Methods
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

# Deep Learning Framework - Keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Conv2D, AveragePooling2D, Flatten
from keras.utils import to_categorical
from keras.backend import clear_session
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.utils import plot_model

# K-fold
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

models = pd.DataFrame()

Using TensorFlow backend.


#  Data

In [2]:
## Data
df = pd.read_csv('data/train.csv')

## Remove colunas constantes
colsToRemove = []
for col in df.columns:
    if df[col].std() == 0:
        colsToRemove.append(col)
df.drop(colsToRemove, axis=1, inplace=True)

## Remove colunas multiplas
colsToRemove = []
columns = df.columns
for i in range(len(columns)-1):
    print(i, end="\r")
    v = df[columns[i]]
    for j in range(i+1,len(columns)):
        m = df.loc[(df[columns[j]]*v) != 0, [columns[j], columns[i]]]
        if m.shape[0] > 0:
            c = m.iloc[0,0]/m.iloc[0,1]
            if np.array_equal(c*v.values,df[columns[j]].values):
                colsToRemove.append(columns[j])
df.drop(colsToRemove, axis=1, inplace=True)

## Remove duplicate rows
df.drop_duplicates(inplace=True)

## Split (X,Y) 
x_data = df.drop(['ID','TARGET'], axis=1)
y_data = df['TARGET'].copy()

x_data.shape

(76020, 287)

# Oversampling: Random Oversampling

## CNN 1 AVG RO

In [3]:
# info
model_name = 'CNN 1 AVG RO Clean'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = RandomOverSampler(random_state=42)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 17, 17, 1)
    x_test = x_test.reshape(x_test.shape[0], 17, 17, 1)
    x_val = x_val.reshape(x_val.shape[0], 17, 17, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(17, 17, 1), activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(243, activation='relu'))
    model.add(Dense(243, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97506 samples, validate on 13684 samples
Epoch 1/10000
97506/97506 [==============================] - 40s - loss: 0.5667 - acc: 0.7118 - val_loss: 0.5735 - val_acc: 0.7071
Epoch 2/10000
97506/97506 [==============================] - 11s - loss: 0.5104 - acc: 0.7537 - val_loss: 0.5974 - val_acc: 0.6950
Epoch 3/10000
97506/97506 [==============================] - 11s - loss: 0.4897 - acc: 0.7634 - val_loss: 0.5682 - val_acc: 0.7091
Epoch 4/10000
97506/97506 [==============================] - 11s - loss: 0.4720 - acc: 0.7735 - val_loss: 0.4452 - val_acc: 0.7634
Epoch 5/10000
97506/97506 [==============================] - 11s - loss: 0.4572 - acc: 0.7826 - val_loss: 0.4611 - val_acc: 0.7724
Epoch 6/10000
97506/97506 [==============================] - 12s - loss: 0.4427 - acc: 0.7916 - val_loss: 0.5570 - val_acc: 0.7231
Epoch 7/10000
97506/97506 [==============================] - 12s - loss: 0.4308 - acc: 0.8000 - val_loss: 0.4654 - val_acc: 0.7838
Epoch 8/10000
97506/97506 [=

97620/97620 [==============================] - 12s - loss: 0.3047 - acc: 0.8652 - val_loss: 0.4670 - val_acc: 0.8284
Epoch 43/10000
97620/97620 [==============================] - 11s - loss: 0.3131 - acc: 0.8643 - val_loss: 0.4021 - val_acc: 0.8526
Epoch 44/10000
97620/97620 [==============================] - 11s - loss: 0.3065 - acc: 0.8658 - val_loss: 0.4134 - val_acc: 0.8459
Epoch 45/10000
97620/97620 [==============================] - 12s - loss: 0.3022 - acc: 0.8668 - val_loss: 0.4366 - val_acc: 0.8242
Epoch 46/10000
97620/97620 [==============================] - 11s - loss: 0.2976 - acc: 0.8695 - val_loss: 0.4701 - val_acc: 0.8256
Epoch 47/10000
97620/97620 [==============================] - 11s - loss: 0.3003 - acc: 0.8688 - val_loss: 0.4393 - val_acc: 0.8162
Epoch 48/10000
97620/97620 [==============================] - 11s - loss: 0.2970 - acc: 0.8694 - val_loss: 0.4816 - val_acc: 0.8242
Epoch 49/10000
97620/97620 [==============================] - 11s - loss: 0.2922 - acc: 0.8

97420/97420 [==============================] - 11s - loss: 0.3266 - acc: 0.8554 - val_loss: 0.4816 - val_acc: 0.7931
Epoch 29/10000
97420/97420 [==============================] - 11s - loss: 0.3223 - acc: 0.8577 - val_loss: 0.3975 - val_acc: 0.8477
Epoch 30/10000
97420/97420 [==============================] - 11s - loss: 0.3236 - acc: 0.8571 - val_loss: 0.4895 - val_acc: 0.7935
Epoch 31/10000
97420/97420 [==============================] - 11s - loss: 0.3167 - acc: 0.8604 - val_loss: 0.3485 - val_acc: 0.8714
Epoch 32/10000
97420/97420 [==============================] - 11s - loss: 0.3182 - acc: 0.8601 - val_loss: 0.5797 - val_acc: 0.7362
Epoch 33/10000
97420/97420 [==============================] - 11s - loss: 0.3174 - acc: 0.8594 - val_loss: 0.5150 - val_acc: 0.7835
Epoch 34/10000
97420/97420 [==============================] - 11s - loss: 0.3093 - acc: 0.8621 - val_loss: 0.4454 - val_acc: 0.8248
Epoch 35/10000
97420/97420 [==============================] - 11s - loss: 0.3079 - acc: 0.8

In [4]:
print(model_name)

metrics_kf

CNN 1 AVG RO Clean


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.945165,0.713945,0.713945,0.546677,0.687466,0.543335,0.815731,00:07:34.890339
KF_2,0.946759,0.732150,0.738794,0.545654,0.708622,0.532244,0.786006,00:09:12.421739
KF_3,0.951992,0.728200,0.728200,0.561715,0.697655,0.575212,0.859774,00:10:18.252338
KF_4,0.942370,0.763664,0.763664,0.560006,0.715827,0.568917,0.842673,00:06:09.296277
KF_5,0.937383,0.736770,0.736770,0.544105,0.701621,0.529847,0.784793,00:05:18.893451
KF_6,0.935501,0.774281,0.777603,0.553276,0.725439,0.549942,0.809129,00:04:58.166026
KF_7,0.929819,0.752897,0.752897,0.565355,0.712058,0.580259,0.859905,00:07:18.484052
KF_8,0.945625,0.719473,0.719473,0.551859,0.702849,0.552683,0.823862,00:07:47.460261
KF_9,0.921903,0.758553,0.758553,0.544304,0.723418,0.520009,0.757269,00:04:21.569784
KF_10,0.937151,0.744499,0.747832,0.559053,0.720170,0.566071,0.836995,00:05:25.882994


## CNN 2 AVG RO

In [5]:
# info
model_name = 'CNN 2 AVG RO Clean'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = RandomOverSampler(random_state=42)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 17, 17, 1)
    x_test = x_test.reshape(x_test.shape[0], 17, 17, 1)
    x_val = x_val.reshape(x_val.shape[0], 17, 17, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(17, 17, 1), activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(96, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97594 samples, validate on 13684 samples
Epoch 1/10000
97594/97594 [==============================] - 13s - loss: 0.6411 - acc: 0.6583 - val_loss: 0.5840 - val_acc: 0.7044
Epoch 2/10000
97594/97594 [==============================] - 12s - loss: 0.6049 - acc: 0.6840 - val_loss: 0.5945 - val_acc: 0.6316
Epoch 3/10000
97594/97594 [==============================] - 12s - loss: 0.5641 - acc: 0.7149 - val_loss: 0.4868 - val_acc: 0.7654
Epoch 4/10000
97594/97594 [==============================] - 12s - loss: 0.5466 - acc: 0.7267 - val_loss: 0.5869 - val_acc: 0.7028
Epoch 5/10000
97594/97594 [==============================] - 12s - loss: 0.5391 - acc: 0.7304 - val_loss: 0.5261 - val_acc: 0.7192
Epoch 6/10000
97594/97594 [==============================] - 12s - loss: 0.5322 - acc: 0.7328 - val_loss: 0.4530 - val_acc: 0.7466
Epoch 7/10000
97594/97594 [==============================] - 12s - loss: 0.5266 - acc: 0.7352 - val_loss: 0.5298 - val_acc: 0.7294
Epoch 8/10000
97594/97594 [=

97516/97516 [==============================] - 12s - loss: 0.5944 - acc: 0.6940 - val_loss: 0.5893 - val_acc: 0.6264
Epoch 3/10000
97516/97516 [==============================] - 12s - loss: 0.5503 - acc: 0.7245 - val_loss: 0.4907 - val_acc: 0.7541
Epoch 4/10000
97516/97516 [==============================] - 12s - loss: 0.5344 - acc: 0.7342 - val_loss: 0.5496 - val_acc: 0.7162
Epoch 5/10000
97516/97516 [==============================] - 12s - loss: 0.5263 - acc: 0.7381 - val_loss: 0.5355 - val_acc: 0.7019
Epoch 6/10000
97516/97516 [==============================] - 12s - loss: 0.5196 - acc: 0.7428 - val_loss: 0.5523 - val_acc: 0.7124
Epoch 7/10000
97516/97516 [==============================] - 12s - loss: 0.5157 - acc: 0.7436 - val_loss: 0.5529 - val_acc: 0.6847
Epoch 8/10000
97516/97516 [==============================] - 12s - loss: 0.5113 - acc: 0.7460 - val_loss: 0.5351 - val_acc: 0.7085
Epoch 9/10000
97516/97516 [==============================] - 14s - loss: 0.5084 - acc: 0.7477 - v

97386/97386 [==============================] - 12s - loss: 0.4511 - acc: 0.7830 - val_loss: 0.4567 - val_acc: 0.7651
Epoch 30/10000
97386/97386 [==============================] - 12s - loss: 0.4468 - acc: 0.7846 - val_loss: 0.4860 - val_acc: 0.7732
Epoch 31/10000
97386/97386 [==============================] - 12s - loss: 0.4443 - acc: 0.7874 - val_loss: 0.4626 - val_acc: 0.7897
Epoch 32/10000
97386/97386 [==============================] - 12s - loss: 0.4412 - acc: 0.7899 - val_loss: 0.4962 - val_acc: 0.7487
Epoch 33/10000
97386/97386 [==============================] - 12s - loss: 0.4410 - acc: 0.7905 - val_loss: 0.5144 - val_acc: 0.7492
Epoch 34/10000
97386/97386 [==============================] - 12s - loss: 0.4389 - acc: 0.7900 - val_loss: 0.4720 - val_acc: 0.7732
Epoch 35/10000
97386/97386 [==============================] - 12s - loss: 0.4350 - acc: 0.7931 - val_loss: 0.5018 - val_acc: 0.7281
Epoch 36/10000
97386/97386 [==============================] - 12s - loss: 0.4348 - acc: 0.7

97408/97408 [==============================] - 12s - loss: 0.4620 - acc: 0.7773 - val_loss: 0.4559 - val_acc: 0.7812
Epoch 25/10000
97408/97408 [==============================] - 12s - loss: 0.4604 - acc: 0.7784 - val_loss: 0.4900 - val_acc: 0.7379
Epoch 26/10000
97408/97408 [==============================] - 12s - loss: 0.4581 - acc: 0.7807 - val_loss: 0.5091 - val_acc: 0.7040
Epoch 27/10000
97408/97408 [==============================] - 12s - loss: 0.4565 - acc: 0.7818 - val_loss: 0.4672 - val_acc: 0.7655
Epoch 28/10000
97408/97408 [==============================] - 12s - loss: 0.4533 - acc: 0.7836 - val_loss: 0.4548 - val_acc: 0.7572
Epoch 29/10000
97408/97408 [==============================] - 12s - loss: 0.4507 - acc: 0.7847 - val_loss: 0.4892 - val_acc: 0.7387
Epoch 30/10000
97408/97408 [==============================] - 12s - loss: 0.4482 - acc: 0.7859 - val_loss: 0.4938 - val_acc: 0.7540
Epoch 31/10000
97408/97408 [==============================] - 12s - loss: 0.4458 - acc: 0.7

In [6]:
print(model_name)

metrics_kf

CNN 2 AVG RO Clean


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.894024,0.768739,0.768739,0.540867,0.712895,0.510150,0.742207,00:10:00.033503
KF_2,0.878147,0.810333,0.810333,0.549205,0.751596,0.525287,0.755360,00:06:08.302946
KF_3,0.885617,0.785004,0.785004,0.542212,0.724901,0.508808,0.734675,00:08:03.088433
KF_4,0.904929,0.763248,0.766571,0.543846,0.724042,0.517276,0.751381,00:13:21.047739
KF_5,0.862239,0.780714,0.780714,0.540325,0.718720,0.503092,0.725862,00:06:12.886755
KF_6,0.898330,0.791245,0.791245,0.548672,0.732276,0.532613,0.776375,00:08:58.475920
KF_7,0.891073,0.786114,0.789436,0.542584,0.733345,0.504706,0.723362,00:11:04.952150
KF_8,0.865458,0.777513,0.777513,0.547906,0.706928,0.540292,0.801105,00:05:46.851521
KF_9,0.904274,0.777966,0.777966,0.544358,0.722642,0.520672,0.758848,00:12:13.401345
KF_10,0.899576,0.762595,0.765928,0.544200,0.727643,0.517112,0.750033,00:12:16.868979


## CNN 3 AVG RO

In [7]:
# info
model_name = 'CNN 3 AVG RO Clean'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = RandomOverSampler(random_state=42)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 17, 17, 1)
    x_test = x_test.reshape(x_test.shape[0], 17, 17, 1)
    x_val = x_val.reshape(x_val.shape[0], 17, 17, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(17, 17, 1), activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(24, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97528 samples, validate on 13684 samples
Epoch 1/10000
97528/97528 [==============================] - 13s - loss: 0.6784 - acc: 0.5819 - val_loss: 0.6567 - val_acc: 0.7157
Epoch 2/10000
97528/97528 [==============================] - 12s - loss: 0.6480 - acc: 0.6612 - val_loss: 0.6210 - val_acc: 0.7170
Epoch 3/10000
97528/97528 [==============================] - 12s - loss: 0.6455 - acc: 0.6642 - val_loss: 0.6304 - val_acc: 0.7171
Epoch 4/10000
97528/97528 [==============================] - 12s - loss: 0.6439 - acc: 0.6645 - val_loss: 0.6720 - val_acc: 0.7148
Epoch 5/10000
97528/97528 [==============================] - 12s - loss: 0.6419 - acc: 0.6645 - val_loss: 0.6860 - val_acc: 0.7076
Epoch 6/10000
97528/97528 [==============================] - 12s - loss: 0.6387 - acc: 0.6662 - val_loss: 0.6417 - val_acc: 0.7099
Epoch 7/10000
97528/97528 [==============================] - 12s - loss: 0.6347 - acc: 0.6689 - val_loss: 0.6094 - val_acc: 0.6969
Epoch 8/10000
97528/97528 [=

97520/97520 [==============================] - 12s - loss: 0.5426 - acc: 0.7293 - val_loss: 0.4762 - val_acc: 0.8109
Epoch 29/10000
97520/97520 [==============================] - 12s - loss: 0.5398 - acc: 0.7315 - val_loss: 0.5519 - val_acc: 0.7451
Epoch 30/10000
97520/97520 [==============================] - 12s - loss: 0.5407 - acc: 0.7317 - val_loss: 0.5191 - val_acc: 0.7801
Epoch 31/10000
97520/97520 [==============================] - 12s - loss: 0.5381 - acc: 0.7305 - val_loss: 0.5574 - val_acc: 0.7424
Epoch 32/10000
97520/97520 [==============================] - 12s - loss: 0.5385 - acc: 0.7309 - val_loss: 0.5602 - val_acc: 0.7540
Epoch 33/10000
97520/97520 [==============================] - 12s - loss: 0.5367 - acc: 0.7334 - val_loss: 0.5578 - val_acc: 0.7136
Epoch 34/10000
97520/97520 [==============================] - 12s - loss: 0.5355 - acc: 0.7322 - val_loss: 0.4993 - val_acc: 0.7982
Epoch 35/10000
97520/97520 [==============================] - 12s - loss: 0.5357 - acc: 0.7

97572/97572 [==============================] - 12s - loss: 0.5595 - acc: 0.7183 - val_loss: 0.6284 - val_acc: 0.6982
Epoch 15/10000
97572/97572 [==============================] - 12s - loss: 0.5583 - acc: 0.7167 - val_loss: 0.4966 - val_acc: 0.7839
Epoch 16/10000
97572/97572 [==============================] - 12s - loss: 0.5572 - acc: 0.7219 - val_loss: 0.4905 - val_acc: 0.8151
Epoch 17/10000
97572/97572 [==============================] - 12s - loss: 0.5538 - acc: 0.7239 - val_loss: 0.5284 - val_acc: 0.7809
Epoch 18/10000
97572/97572 [==============================] - 12s - loss: 0.5497 - acc: 0.7273 - val_loss: 0.7235 - val_acc: 0.6009
Epoch 19/10000
97572/97572 [==============================] - 12s - loss: 0.5484 - acc: 0.7290 - val_loss: 0.5210 - val_acc: 0.7737
Epoch 20/10000
97572/97572 [==============================] - 12s - loss: 0.5478 - acc: 0.7312 - val_loss: 0.5741 - val_acc: 0.7170
Epoch 21/10000
97572/97572 [==============================] - 12s - loss: 0.5452 - acc: 0.7

97450/97450 [==============================] - 12s - loss: 0.6161 - acc: 0.6807 - val_loss: 0.6822 - val_acc: 0.6469
Epoch 7/10000
97450/97450 [==============================] - 12s - loss: 0.6015 - acc: 0.6899 - val_loss: 0.6604 - val_acc: 0.6008
Epoch 8/10000
97450/97450 [==============================] - 12s - loss: 0.5749 - acc: 0.7189 - val_loss: 0.4946 - val_acc: 0.8050
Epoch 9/10000
97450/97450 [==============================] - 12s - loss: 0.5643 - acc: 0.7304 - val_loss: 0.5632 - val_acc: 0.7526
Epoch 10/10000
97450/97450 [==============================] - 12s - loss: 0.5594 - acc: 0.7302 - val_loss: 0.5639 - val_acc: 0.7204
Epoch 11/10000
97450/97450 [==============================] - 12s - loss: 0.5545 - acc: 0.7344 - val_loss: 0.4485 - val_acc: 0.8009
Epoch 12/10000
97450/97450 [==============================] - 12s - loss: 0.5518 - acc: 0.7352 - val_loss: 0.4270 - val_acc: 0.8394
Epoch 13/10000
97450/97450 [==============================] - 12s - loss: 0.5475 - acc: 0.7382

In [8]:
print(model_name)

metrics_kf

CNN 3 AVG RO Clean


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.810143,0.795626,0.795626,0.549366,0.717696,0.540909,0.797317,00:10:58.973911
KF_2,0.822635,0.818722,0.818722,0.546010,0.751413,0.510332,0.727476,00:09:12.029418
KF_3,0.827695,0.799054,0.799054,0.534636,0.712787,0.465000,0.650224,00:13:04.929618
KF_4,0.845871,0.803059,0.803059,0.544128,0.736889,0.510483,0.733228,00:16:13.645847
KF_5,0.822771,0.794686,0.794686,0.536140,0.720715,0.469206,0.656275,00:06:36.428791
KF_6,0.826730,0.812180,0.812180,0.541147,0.732114,0.497073,0.708761,00:06:39.978570
KF_7,0.824705,0.800520,0.803842,0.542514,0.733139,0.504464,0.722968,00:08:22.414639
KF_8,0.824291,0.791417,0.791417,0.545913,0.723458,0.526313,0.768614,00:05:15.028267
KF_9,0.835357,0.802555,0.802555,0.545607,0.720883,0.526678,0.770820,00:06:50.586967
KF_10,0.831269,0.801464,0.801464,0.558169,0.729281,0.561953,0.826865,00:09:03.765982


# Oversampling: SMOTE

## CNN 1 AVG SM

In [9]:
# info
model_name = 'CNN 1 AVG SM Clean'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = SMOTE(kind='borderline2',k_neighbors=5, random_state=42, n_jobs=4)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 17, 17, 1)
    x_test = x_test.reshape(x_test.shape[0], 17, 17, 1)
    x_val = x_val.reshape(x_val.shape[0], 17, 17, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(17, 17, 1), activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(243, activation='relu'))
    model.add(Dense(243, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97539 samples, validate on 13684 samples
Epoch 1/10000
97539/97539 [==============================] - 12s - loss: 0.4384 - acc: 0.8082 - val_loss: 0.2163 - val_acc: 0.8948
Epoch 2/10000
97539/97539 [==============================] - 11s - loss: 0.3388 - acc: 0.8614 - val_loss: 0.1576 - val_acc: 0.9562
Epoch 3/10000
97539/97539 [==============================] - 11s - loss: 0.3136 - acc: 0.8722 - val_loss: 1.2725 - val_acc: 0.4070
Epoch 4/10000
97539/97539 [==============================] - 11s - loss: 0.2975 - acc: 0.8799 - val_loss: 0.4596 - val_acc: 0.8007
Epoch 5/10000
97539/97539 [==============================] - 12s - loss: 0.2864 - acc: 0.8860 - val_loss: 0.4073 - val_acc: 0.8188
Epoch 6/10000
97539/97539 [==============================] - 11s - loss: 0.2780 - acc: 0.8903 - val_loss: 0.3388 - val_acc: 0.8514
Epoch 7/10000
97539/97539 [==============================] - 11s - loss: 0.2693 - acc: 0.8927 - val_loss: 0.2776 - val_acc: 0.8793
Epoch 8/10000
97539/97539 [=

97530/97530 [==============================] - 11s - loss: 0.2371 - acc: 0.9053 - val_loss: 0.2812 - val_acc: 0.8815
Epoch 25/10000
97530/97530 [==============================] - 11s - loss: 0.2336 - acc: 0.9063 - val_loss: 0.3772 - val_acc: 0.8481
Epoch 26/10000
97472/97530 [============================>.] - ETA: 0sKF_6
Train on 97455 samples, validate on 13684 samples
Epoch 1/10000
97455/97455 [==============================] - 12s - loss: 0.4574 - acc: 0.7975 - val_loss: 0.5663 - val_acc: 0.7368
Epoch 2/10000
97455/97455 [==============================] - 11s - loss: 0.3431 - acc: 0.8565 - val_loss: 0.4253 - val_acc: 0.8216
Epoch 3/10000
97455/97455 [==============================] - 11s - loss: 0.3179 - acc: 0.8685 - val_loss: 0.4058 - val_acc: 0.8223
Epoch 4/10000
97455/97455 [==============================] - 11s - loss: 0.3012 - acc: 0.8777 - val_loss: 0.3612 - val_acc: 0.8419
Epoch 5/10000
97455/97455 [==============================] - 11s - loss: 0.2882 - acc: 0.8829 - val_los

97504/97504 [==============================] - 11s - loss: 0.2893 - acc: 0.8832 - val_loss: 0.3635 - val_acc: 0.8429
Epoch 6/10000
97504/97504 [==============================] - 11s - loss: 0.2793 - acc: 0.8880 - val_loss: 0.3162 - val_acc: 0.8632
Epoch 7/10000
97504/97504 [==============================] - 11s - loss: 0.2709 - acc: 0.8920 - val_loss: 0.3645 - val_acc: 0.8427
Epoch 8/10000
97504/97504 [==============================] - 11s - loss: 0.2651 - acc: 0.8941 - val_loss: 0.2581 - val_acc: 0.8852
Epoch 9/10000
97504/97504 [==============================] - 11s - loss: 0.2602 - acc: 0.8973 - val_loss: 0.3495 - val_acc: 0.8446
Epoch 10/10000
97504/97504 [==============================] - 11s - loss: 0.2557 - acc: 0.8992 - val_loss: 0.2633 - val_acc: 0.8843
Epoch 11/10000
97504/97504 [==============================] - 11s - loss: 0.2520 - acc: 0.9011 - val_loss: 0.3193 - val_acc: 0.8583
Epoch 12/10000
97504/97504 [==============================] - 11s - loss: 0.2472 - acc: 0.9022 

In [10]:
print(model_name)

metrics_kf

CNN 1 AVG SM Clean


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.968054,0.770692,0.770692,0.562535,0.688826,0.577704,0.867289,00:04:29.466772
KF_2,0.966646,0.787534,0.790856,0.558830,0.723580,0.564704,0.833092,00:04:26.773022
KF_3,0.972357,0.773615,0.773615,0.571328,0.702328,0.591377,0.877927,00:05:01.789749
KF_4,0.969145,0.777552,0.777552,0.562835,0.708342,0.575935,0.855827,00:04:49.947710
KF_5,0.967514,0.764043,0.764043,0.561702,0.698837,0.575037,0.858984,00:05:05.079617
KF_6,0.973466,0.773432,0.773432,0.573158,0.697310,0.594391,0.883583,00:06:12.760052
KF_7,0.972532,0.783421,0.786743,0.562877,0.710774,0.575675,0.854380,00:05:10.170384
KF_8,0.972313,0.768856,0.772178,0.563870,0.693664,0.579574,0.867403,00:05:02.287215
KF_9,0.970964,0.783008,0.783008,0.573819,0.707541,0.595113,0.880279,00:05:12.152521
KF_10,0.971331,0.755258,0.755258,0.561032,0.719074,0.570654,0.844099,00:04:37.686694


## CNN 2 AVG SM

In [11]:
# info
model_name = 'CNN 2 AVG SM Clean'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = SMOTE(kind='borderline2',k_neighbors=5, random_state=42, n_jobs=4)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 17, 17, 1)
    x_test = x_test.reshape(x_test.shape[0], 17, 17, 1)
    x_val = x_val.reshape(x_val.shape[0], 17, 17, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(17, 17, 1), activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(96, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97564 samples, validate on 13684 samples
Epoch 1/10000
97564/97564 [==============================] - 12s - loss: 0.5664 - acc: 0.7387 - val_loss: 0.5068 - val_acc: 0.7219
Epoch 2/10000
97564/97564 [==============================] - 12s - loss: 0.4941 - acc: 0.7778 - val_loss: 0.3229 - val_acc: 0.8684
Epoch 3/10000
97564/97564 [==============================] - 12s - loss: 0.4119 - acc: 0.8247 - val_loss: 0.3048 - val_acc: 0.8682
Epoch 4/10000
97564/97564 [==============================] - 11s - loss: 0.3787 - acc: 0.8417 - val_loss: 0.3026 - val_acc: 0.8659
Epoch 5/10000
97564/97564 [==============================] - 12s - loss: 0.3593 - acc: 0.8505 - val_loss: 0.5334 - val_acc: 0.7539
Epoch 6/10000
97564/97564 [==============================] - 12s - loss: 0.3467 - acc: 0.8565 - val_loss: 0.3159 - val_acc: 0.8583
Epoch 7/10000
97564/97564 [==============================] - 12s - loss: 0.3417 - acc: 0.8593 - val_loss: 0.3544 - val_acc: 0.8449
Epoch 8/10000
97564/97564 [=

97501/97501 [==============================] - 11s - loss: 0.3168 - acc: 0.8698 - val_loss: 0.3936 - val_acc: 0.8155
Epoch 15/10000
97501/97501 [==============================] - 12s - loss: 0.3124 - acc: 0.8719 - val_loss: 0.3170 - val_acc: 0.8578
Epoch 16/10000
97501/97501 [==============================] - 11s - loss: 0.3124 - acc: 0.8728 - val_loss: 0.3005 - val_acc: 0.8726
Epoch 17/10000
97501/97501 [==============================] - 12s - loss: 0.3109 - acc: 0.8724 - val_loss: 0.3241 - val_acc: 0.8612
Epoch 18/10000
97501/97501 [==============================] - 12s - loss: 0.3085 - acc: 0.8741 - val_loss: 0.2994 - val_acc: 0.8693
Epoch 19/10000
97501/97501 [==============================] - 12s - loss: 0.3070 - acc: 0.8746 - val_loss: 0.3085 - val_acc: 0.8653
Epoch 20/10000
97501/97501 [==============================] - 11s - loss: 0.3060 - acc: 0.8751 - val_loss: 0.3276 - val_acc: 0.8604
Epoch 21/10000
97501/97501 [==============================] - 11s - loss: 0.3031 - acc: 0.8

97569/97569 [==============================] - 11s - loss: 0.3734 - acc: 0.8435 - val_loss: 0.5313 - val_acc: 0.7667
Epoch 5/10000
97569/97569 [==============================] - 11s - loss: 0.3599 - acc: 0.8501 - val_loss: 0.3668 - val_acc: 0.8339
Epoch 6/10000
97569/97569 [==============================] - 11s - loss: 0.3531 - acc: 0.8521 - val_loss: 0.3829 - val_acc: 0.8349
Epoch 7/10000
97569/97569 [==============================] - 11s - loss: 0.3462 - acc: 0.8555 - val_loss: 0.6416 - val_acc: 0.7268
Epoch 8/10000
97569/97569 [==============================] - 11s - loss: 0.3399 - acc: 0.8576 - val_loss: 0.4957 - val_acc: 0.7795
Epoch 9/10000
97569/97569 [==============================] - 11s - loss: 0.3361 - acc: 0.8595 - val_loss: 0.4259 - val_acc: 0.8134
Epoch 10/10000
97569/97569 [==============================] - 11s - loss: 0.3320 - acc: 0.8625 - val_loss: 0.4544 - val_acc: 0.8035
Epoch 11/10000
97569/97569 [==============================] - 11s - loss: 0.3284 - acc: 0.8638 -

In [12]:
print(model_name)

metrics_kf

CNN 2 AVG SM Clean


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.952528,0.793324,0.796646,0.555561,0.706883,0.560752,0.834671,00:06:29.244792
KF_2,0.951445,0.817170,0.820492,0.574995,0.747019,0.594149,0.862817,00:05:49.038281
KF_3,0.959350,0.777937,0.777937,0.564806,0.705464,0.580015,0.862536,00:09:23.151749
KF_4,0.950396,0.801073,0.801073,0.565567,0.732764,0.577754,0.847672,00:05:25.903938
KF_5,0.954884,0.783081,0.783081,0.561646,0.709473,0.573420,0.851881,00:06:03.382000
KF_6,0.953754,0.807240,0.810562,0.567352,0.732249,0.581444,0.852802,00:06:48.387696
KF_7,0.958277,0.791241,0.794563,0.567195,0.702723,0.584526,0.869508,00:07:48.502874
KF_8,0.950203,0.764589,0.764589,0.556112,0.705330,0.562354,0.837806,00:06:37.169201
KF_9,0.948863,0.777785,0.781118,0.558072,0.723481,0.563071,0.831075,00:06:15.985186
KF_10,0.946718,0.776740,0.780074,0.556900,0.715125,0.562177,0.833443,00:05:13.196033


## CNN 3 AVG SM

In [14]:
# info
model_name = 'CNN 3 AVG SM Clean'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = SMOTE(kind='borderline2',k_neighbors=5, random_state=42, n_jobs=4)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 17, 17, 1)
    x_test = x_test.reshape(x_test.shape[0], 17, 17, 1)
    x_val = x_val.reshape(x_val.shape[0], 17, 17, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(17, 17, 1), activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(24, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97589 samples, validate on 13684 samples
Epoch 1/10000
97589/97589 [==============================] - 16s - loss: 0.6529 - acc: 0.6405 - val_loss: 0.6634 - val_acc: 0.7134
Epoch 2/10000
97589/97589 [==============================] - 13s - loss: 0.5988 - acc: 0.7293 - val_loss: 0.5747 - val_acc: 0.7127
Epoch 3/10000
97589/97589 [==============================] - 13s - loss: 0.5718 - acc: 0.7462 - val_loss: 0.5579 - val_acc: 0.6851
Epoch 4/10000
97589/97589 [==============================] - 13s - loss: 0.5382 - acc: 0.7562 - val_loss: 0.4397 - val_acc: 0.7257
Epoch 5/10000
97589/97589 [==============================] - 13s - loss: 0.5166 - acc: 0.7657 - val_loss: 0.7732 - val_acc: 0.6747
Epoch 6/10000
97589/97589 [==============================] - 13s - loss: 0.5015 - acc: 0.7705 - val_loss: 0.4278 - val_acc: 0.7322
Epoch 7/10000
97589/97589 [==============================] - 13s - loss: 0.4870 - acc: 0.7787 - val_loss: 0.3697 - val_acc: 0.8454
Epoch 8/10000
97589/97589 [=

97585/97585 [==============================] - 12s - loss: 0.3973 - acc: 0.8314 - val_loss: 0.4774 - val_acc: 0.8161
Epoch 15/10000
97585/97585 [==============================] - 12s - loss: 0.3927 - acc: 0.8346 - val_loss: 0.3884 - val_acc: 0.8560
Epoch 16/10000
97585/97585 [==============================] - 12s - loss: 0.3901 - acc: 0.8352 - val_loss: 0.3970 - val_acc: 0.8601
Epoch 17/10000
97585/97585 [==============================] - 12s - loss: 0.3883 - acc: 0.8363 - val_loss: 0.5613 - val_acc: 0.7925
Epoch 18/10000
97585/97585 [==============================] - 12s - loss: 0.3841 - acc: 0.8385 - val_loss: 0.3704 - val_acc: 0.8612
Epoch 19/10000
97585/97585 [==============================] - 12s - loss: 0.3826 - acc: 0.8376 - val_loss: 0.3963 - val_acc: 0.8550
Epoch 20/10000
97585/97585 [==============================] - 12s - loss: 0.3816 - acc: 0.8409 - val_loss: 0.4889 - val_acc: 0.8267
Epoch 21/10000
97585/97585 [==============================] - 12s - loss: 0.3792 - acc: 0.8

97507/97507 [==============================] - 12s - loss: 0.3719 - acc: 0.8504 - val_loss: 0.3266 - val_acc: 0.8701
Epoch 27/10000
97507/97507 [==============================] - 12s - loss: 0.3684 - acc: 0.8509 - val_loss: 0.4742 - val_acc: 0.7997
Epoch 28/10000
97507/97507 [==============================] - 12s - loss: 0.3694 - acc: 0.8522 - val_loss: 0.2932 - val_acc: 0.8744
Epoch 29/10000
97507/97507 [==============================] - 12s - loss: 0.3671 - acc: 0.8531 - val_loss: 0.4230 - val_acc: 0.8311
Epoch 30/10000
97507/97507 [==============================] - 12s - loss: 0.3658 - acc: 0.8525 - val_loss: 0.5429 - val_acc: 0.7895
Epoch 31/10000
97507/97507 [==============================] - 12s - loss: 0.3662 - acc: 0.8529 - val_loss: 0.3236 - val_acc: 0.8560
Epoch 32/10000
97507/97507 [==============================] - 12s - loss: 0.3657 - acc: 0.8535 - val_loss: 0.3415 - val_acc: 0.8557
Epoch 33/10000
97507/97507 [==============================] - 12s - loss: 0.3642 - acc: 0.8

97438/97438 [==============================] - 12s - loss: 0.3454 - acc: 0.8537 - val_loss: 0.2654 - val_acc: 0.8751
Epoch 27/10000
97438/97438 [==============================] - 12s - loss: 0.3407 - acc: 0.8570 - val_loss: 0.3289 - val_acc: 0.8483
Epoch 28/10000
97438/97438 [==============================] - 12s - loss: 0.3417 - acc: 0.8553 - val_loss: 0.3097 - val_acc: 0.8570
Epoch 29/10000
97438/97438 [==============================] - 12s - loss: 0.3418 - acc: 0.8555 - val_loss: 0.3971 - val_acc: 0.8172
Epoch 30/10000
97438/97438 [==============================] - 12s - loss: 0.3396 - acc: 0.8565 - val_loss: 0.2964 - val_acc: 0.8363
Epoch 31/10000
97438/97438 [==============================] - 12s - loss: 0.3395 - acc: 0.8561 - val_loss: 0.2873 - val_acc: 0.8677
Epoch 32/10000
97438/97438 [==============================] - 10s    
KF_9
Train on 97572 samples, validate on 13684 samples
Epoch 1/10000
97572/97572 [==============================] - 12s - loss: 0.6441 - acc: 0.6456 - va

In [15]:
print(model_name)

metrics_kf

CNN 3 AVG SM Clean


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.938338,0.780862,0.780862,0.558157,0.693966,0.568822,0.852690,00:13:05.675209
KF_2,0.942117,0.810778,0.810778,0.564332,0.740858,0.573701,0.838748,00:10:25.964082
KF_3,0.939249,0.782440,0.782440,0.569091,0.690546,0.588414,0.879768,00:12:31.809886
KF_4,0.932397,0.798547,0.798547,0.560823,0.721445,0.569642,0.841226,00:10:20.373491
KF_5,0.936947,0.776853,0.776853,0.556183,0.697469,0.564092,0.844120,00:09:09.554630
KF_6,0.941153,0.799379,0.802701,0.560161,0.721805,0.568109,0.838858,00:09:01.163874
KF_7,0.934900,0.805252,0.805252,0.565541,0.715877,0.580135,0.858064,00:06:43.824637
KF_8,0.938347,0.773111,0.773111,0.551869,0.690877,0.555730,0.834517,00:06:43.196698
KF_9,0.935343,0.799631,0.802965,0.553962,0.729624,0.550779,0.809104,00:05:57.994335
KF_10,0.939455,0.789654,0.789654,0.562784,0.718685,0.574392,0.849493,00:09:17.324640


# Metrics

In [16]:
models

,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
CNN 1 AVG RO Clean,0.939367,0.742443,0.743773,0.553200,0.709513,0.551852,0.817614,00:06:50.531726
CNN 2 AVG RO Clean,0.888367,0.780347,0.781345,0.544418,0.725499,0.518001,0.751921,00:09:24.590929
CNN 3 AVG RO Clean,0.827147,0.801928,0.802261,0.544363,0.727837,0.511241,0.736255,00:09:13.778201
CNN 1 AVG SM Clean,0.970432,0.773741,0.774738,0.565199,0.705028,0.580016,0.862287,00:05:00.811373
CNN 2 AVG SM Clean,0.952642,0.789018,0.791013,0.562821,0.718051,0.573966,0.848421,00:06:35.396175
CNN 3 AVG SM Clean,0.937825,0.791651,0.792316,0.560290,0.712115,0.569382,0.844659,00:09:19.688148
